#### Calculando Distância e Duração de Rotas Utilizando Diferentes Meios de Transporte
Calculating Distance and Duration of Routes Using Different Means of Transportation

In [113]:
# Importando Bibliotecas | Importing Libraries
import requests
import time
import json
import folium
import imgkit
import pdfkit
from fpdf import FPDF
import os
from dotenv import load_dotenv

In [ ]:
# Chave da API | API Key
load_dotenv() # Lê e carrega variáveis de .env | # Reads and loads variables from .env
api_key = os.getenv('ors_api_key')
if not api_key:
    raise ValueError("Chave de API não encontrada | API key not found")

# Para obter sua chave, visite https://account.heigit.org/login e crie uma conta. 
# Assim que sua conta for ativada, vá para o seu Painel (Dashboard), copie a longa sequência de caracteres que aparece em Chave Básica (Basic Key) e a atribua a variável ors_api_key.

# To get your key, visit https://account.heigit.org/login and create an account.
# Once your account is activated, go to your Dashboard, copy the long string that appears under Basic Key, and assign it to the ors_api_key variable.

In [ ]:
# Função para calcular distância e duração de rotas utilizando diferentes meios de transporte
# Function to calculate route distance and duration using different modes of transport

def get_route_info(source_coords, dest_coords, profile, api_key): # coordenadas devem estar em Graus Decimais | coordinates must be in Decimal Degrees (DD) 
    params = {
        'api_key': api_key, # API Key
        'start': '{},{}'.format(source_coords[1], source_coords[0]), # ORS exige formato lon,lat | ORS requires lon, lat format
        'end': '{},{}'.format(dest_coords[1], dest_coords[0])
    }

    url = f"https://api.openrouteservice.org/v2/directions/{profile}"

    try:
        response = requests.get(url, params=params)
        response.raise_for_status() # Verifica se a requisição foi bem-sucedida | # Checks if the request was successful

        data = response.json()
        summary = data['features'][0]['properties']['summary']

        distance_km = summary.get('distance', 0) / 1000
        duration_seconds = summary.get('duration', 0) # Duração em segundos | Duration in seconds

        # Converter segundos para HH:MM:SS
        hours = int(duration_seconds // 3600)
        minutes = int((duration_seconds % 3600) // 60)
        seconds = int(duration_seconds % 60)
        duration_formatted_str = f"{hours:02d}:{minutes:02d}:{seconds:02d}"
        
        return distance_km, duration_formatted_str
    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição para o perfil '{profile}': {e}")
        print(f"Request error for the profile '{profile}': {e}")
        if response is not None:
             print(f"Resposta da API: {response.text}")
             print(f"API Response: {response.text}")
        return None, None, None
    except (KeyError, IndexError):
        print(f"Erro ao analisar dados para o perfil '{profile}'. Resposta: {response.text}")
        print(f"Error parsing data for the profile '{profile}'. Response: {response.text}")
        return None, None, None

In [ ]:
# Configuração dos Diretórios | Setting up Directories
output_base_dir = os.path.join('..','outputs') 
figures_dir = os.path.join(output_base_dir, "figures")
results_dir = os.path.join(output_base_dir, "results")

# Verifica se os diretórios existem e os cria se não existirem
# Checks if the directories exist and creates them if they don't
os.makedirs(figures_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

In [ ]:
# Função para criar um mapa com a rota e salvar como arquivo JPG
# Function to create a map with the route and save it as a JPG file
def plot_route_on_map(source_coords, dest_coords, route_geometry, profile_name, city_name, folder_path=figures_dir):
    
    # Inverter as coordenadas para o formato esperado pelo Folium (lat, lon)
    # Invert the coordinates to the format expected by Folium (lat, lon)
    geometry_folium = [list(reversed(coords)) for coords in route_geometry]

    # Criar o mapa centralizado no ponto de origem
    # Create the map centered on the origin point
    m = folium.Map(location=[source_coords[0], source_coords[1]], zoom_start=8)

    # Adicionar a rota ao mapa
    # Add route to the map
    folium.PolyLine(
        locations=geometry_folium,
        color='blue',
        weight=5,
        opacity=0.7
    ).add_to(m)

    # Adicionar marcadores para origem e destino
    # Add markers for origin and destination
    folium.Marker(
        location=source_coords,
        popup=f"Origem: Niterói",
        icon=folium.Icon(color='green', icon='play', prefix='fa')
    ).add_to(m)

    folium.Marker(
        location=dest_coords,
        popup=f"Destino: {city_name}",
        icon=folium.Icon(color='red', icon='stop', prefix='fa')
    ).add_to(m)

    # Salvar o mapa como um arquivo HTML temporário
    # Save the map as a temporary HTML file
    map_html_path = os.path.join(folder_path, f"mapa_temp_{profile_name}_{city_name}.html")
    m.save(map_html_path)

    # Converter o arquivo HTML para JPG
    # Convert the HTML file to JPG
    map_jpg_path = os.path.join(folder_path, f"mapa_{profile_name}_{city_name}.jpg")
    imgkit.from_file(map_html_path, map_jpg_path)

    # Remover o arquivo HTML temporário
    # Remove the temporary HTML file
    os.remove(map_html_path)

    return map_jpg_path

In [ ]:
# Função para gerar um arquivo PDF com texto e imagens
# Function to generate a PDF file with text and images

def generate_pdf_report(text_content, image_paths, pdf_filename="relatorio_rotas.pdf", folder_path=results_dir):
    pdf_output_path = os.path.join(folder_path, pdf_filename)
    
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    
    # Título
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(200, 10, txt="Cálculo de Distância e Duração com Diferentes Meios de Transporte", ln=True, align='C')
    pdf.ln(10)
    
    # Texto do console
    pdf.set_font("Courier", size=10) # Usa uma fonte de espaçamento fixo para o console
    for line in text_content.split('\n'):
        pdf.multi_cell(0, 5, line)
    
    # Adiciona as imagens dos mapas
    pdf.add_page()
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(200, 10, txt="Visualização das Rotas", ln=True, align='C')
    pdf.ln(10)

    for img_path in image_paths:
        try:
            pdf.image(img_path, w=180)
            pdf.ln(5)
        except Exception as e:
            print(f"Não foi possível adicionar a imagem {img_path} ao PDF: {e}")
            
    pdf.output(pdf_output_path)
    print(f"\nRelatório em PDF gerado com sucesso: {pdf_output_path}")


In [ ]:
# Origem | Source
source = (-22.890656, -43.123864) # Caminho Niemeyer, Niterói (RJ)

# Destinos | Destination
destination_cities = {
    'Rio de Janeiro': (-22.9519, -43.2105), # Cristo Redentor, Rio de Janeiro (RJ)
    'Petropolis': (-22.5050, -43.1812), # Palácio de Cristal, Petrópolis (RJ)
    'Paraty': (-23.2195, -44.7170) # Casa da Cultura de Paraty, Paraty (RJ)
}

In [ ]:
# Calculando rotas para todos os destinos utilizando meios de transporte diferentes
# Calculating routes for all destinations using different means of transport

print("--- Calculando rotas para todos os destinos utilizando meios de transporte diferentes ---")

niteroi_coords = source

means_of_transportation = {
    'driving-car': 'carro',
    'foot-walking': 'caminhada',
    'cycling-regular': 'bicicleta regular'
}

for profile_key, profile_name_pt in means_of_transportation.items():
    print(f"\n--- Meio de Transporte: {profile_name_pt.capitalize()} ({profile_key}) ---")
    for city_name, coords in destination_cities.items():
        print(f"  Calculando rota de Niterói para {city_name}...")
        
        # A função duração formatada como string | # The function duration formatted as a string
        distance_km, duration_formatted_str = get_route_info(niteroi_coords, coords, profile_key, api_key)
        
        if distance_km is not None:
            print(f"    Distância: {distance_km:.2f} km")
            print(f"    Duração: {duration_formatted_str} h") # Exibe a string formatada | Displays the formatted string
        else:
            print(f"    Não foi possível obter a rota para {city_name} com o perfil {profile_key}.")
        
        time.sleep(2) # limitação de taxa | rate limit
        print("  " + "-" * 25)

print("\n --- Finalizado | Finish ---")

--- Calculando rotas para todos os destinos utilizando meios de transporte diferentes ---

--- Meio de Transporte: Carro (driving-car) ---
  Calculando rota de Niterói para Rio de Janeiro...
    Distância: 33.78 km
    Duração: 00:49:01 h
  -------------------------
  Calculando rota de Niterói para Petropolis...
    Distância: 75.62 km
    Duração: 01:10:23 h
  -------------------------
  Calculando rota de Niterói para Paraty...
    Distância: 256.53 km
    Duração: 03:54:40 h
  -------------------------

--- Meio de Transporte: Caminhada (foot-walking) ---
  Calculando rota de Niterói para Rio de Janeiro...
    Distância: 16.85 km
    Duração: 02:52:53 h
  -------------------------
  Calculando rota de Niterói para Petropolis...
    Distância: 82.26 km
    Duração: 15:57:47 h
  -------------------------
  Calculando rota de Niterói para Paraty...
    Distância: 277.25 km
    Duração: 54:57:43 h
  -------------------------

--- Meio de Transporte: Bicicleta regular (cycling-regular) 

In [ ]:
# Utilização das funções